# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

# cities.csv
csv_path = '../WeatherPy/output_data/cities.csv'


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
weather_df = pd.read_csv(csv_path)
weather_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)
locations = weather_df[['Lat', 'Lng']]
weights = weather_df['Humidity']

# set center and zoom
fig = gmaps.figure(center=(0, 0), zoom_level=2)
# set max intensity to max humidity
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, max_intensity=weather_df['Humidity'].max()))
fig


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
ideal_max_temp = 75
ideal_min_temp = 65
ideal_max_wind_speed = 5
ideal_max_cloudiness = 5

ideal_weather_df = weather_df.loc[(weather_df['Max_Temp'] < ideal_max_temp) &
                                  (weather_df['Max_Temp'] > ideal_min_temp) &
                                  (weather_df['Wind_Speed'] < ideal_max_wind_speed) & 
                                  (weather_df['Cloudiness'] < ideal_max_cloudiness)] 
ideal_weather_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# static target params
target_search = 'Hotel'
target_radius = 5000
target_type = 'lodging'

# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [ ]:
d = []
for index, row in ideal_weather_df.iterrows():
    params['location'] = f"{row['Lat']}, {row['Lng']}"
    data = requests.get(base_url, params=params).json()
    if data['status'] == "ZERO_RESULTS":
        continue
    d.append(
        {
            'Hotel_Name': data['results'][0]['name'],
            'City': row['City'],
            'Country': row['Country'],
            'Lat': data['results'][0]['geometry']['location']['lat'],
            'Lng': data['results'][0]['geometry']['location']['lng']
        }
    )

hotel_df = pd.DataFrame(d)
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<strong>Name</strong><dd>{Hotel_Name}</dd>
<strong>City</strong><dd>{City}</dd>
<strong>Country</strong><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig